### Step 1: 读取数据

In [1]:
import tensorflow as tf
import time
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("/tmp/data", one_hot=True)

print "Training data size: ", mnist.train.num_examples
print "Validating data size: ", mnist.validation.num_examples
print "Testing data size: ", mnist.test.num_examples
print "Example training data: ", mnist.train.images[0]
print "Example training data label: ", mnist.train.labels[0]

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Training data size:  55000
Validating data size:  5000
Testing data size:  10000
Example training data:  [ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.        

### Step 2: 建立神经网络

In [2]:
def nn_layer(input_tensor, input_dim, output_dim, layer_name, act=tf.nn.relu):
    weights = tf.Variable(tf.truncated_normal([input_dim, output_dim], stddev=0.1, seed = 2))
    biases = tf.Variable(tf.constant(0.1, shape=[output_dim]))
    activations = act(tf.matmul(input_tensor, weights) + biases)
    return activations

x = tf.placeholder(tf.float32, [None, 784], name='x-input')
y_ = tf.placeholder(tf.float32, [None, 10], name='y-input')

hidden_nodes = 500
hidden1 = nn_layer(x, 784, hidden_nodes, 'layer1')
y = nn_layer(hidden1, hidden_nodes, 10, 'layer2', act=tf.nn.softmax)
print("Network created!")

Network created!


### Step 3: 指定训练过程

In [3]:
cross_entropy = -tf.reduce_mean(y_ * tf.log(tf.clip_by_value(y, 1e-10, 1.0)))
train_step = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("Training & eval step setup!")

Training & eval step setup!


### Step 4: 训练模型

In [4]:
sess = tf.InteractiveSession()

tf.initialize_all_variables().run()
validate_feed = {x: mnist.validation.images, y_: mnist.validation.labels}
test_feed = {x: mnist.test.images, y_: mnist.test.labels}

def feed_dict(train):
    """Make a TensorFlow feed_dict: maps data onto Tensor placeholders."""
    if train:
        xs, ys = mnist.train.next_batch(100)
    else:
        xs, ys = mnist.test.images, mnist.test.labels
    return {x: xs, y_: ys}

STEPS = 500
for i in range(STEPS):
    sess.run(train_step, feed_dict=feed_dict(True))
    if i % 100 == 0:
        validate_acc = sess.run(accuracy, feed_dict=validate_feed)
        test_acc = sess.run(accuracy, feed_dict=test_feed)
        print("After %d training step(s), validation accuracy = %g, test accuracy = %g" %  
              (i, validate_acc, test_acc))
        
test_acc = sess.run(accuracy, feed_dict=test_feed)
print("After %d training step(s), test accuracy = %g" %  (STEPS, test_acc))
sess.close()

After 0 training step(s), validation accuracy = 0.2188, test accuracy = 0.2096
After 100 training step(s), validation accuracy = 0.9168, test accuracy = 0.9113
After 200 training step(s), validation accuracy = 0.9382, test accuracy = 0.934
After 300 training step(s), validation accuracy = 0.9528, test accuracy = 0.9483
After 400 training step(s), validation accuracy = 0.9538, test accuracy = 0.9535
After 500 training step(s), test accuracy = 0.9557
